# 2018 General Election Midterm Candidate Scraper

In [122]:
from urllib.request import urlopen
import re # Regex Library
import pandas as pd

In [2]:
def get_html_from_url(url): 
    return urlopen(url).read().decode("utf-8")

In [77]:
states = []
with open('states.txt', 'r') as f:
    for state in f:
        states.append(state[:-1].replace(' ', '_'))

In [78]:
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New_Hampshire',
 'New_Jersey',
 'New_Mexico',
 'New_York',
 'North_Carolina',
 'North_Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode_Island',
 'South_Carolina',
 'South_Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West_Virginia',
 'Wisconsin',
 'Wyoming']

## House of Representatives

### Get Ballotpedia HTML for each state

In [84]:
state_general_election_html = {}
for state in states:
    html = get_html_from_url("https://ballotpedia.org/United_States_House_of_Representatives_elections_in_{},_2018".format(state))
    state_general_election_html[state] = re.findall(r"District \d{1,2}.+?General election candidates.+?Primary", html, re.DOTALL)

In [80]:
len(state_html)

50

In [6]:
ca = get_html_from_url("https://ballotpedia.org/United_States_House_of_Representatives_elections_in_California,_2018")

In [16]:
re.DOTALL

<RegexFlag.DOTALL: 16>

In [63]:
re.findall(r"District \d{1,2}", ca, re.DOTALL)

['District 1',
 'District 2',
 'District 3',
 'District 4',
 'District 5',
 'District 6',
 'District 7',
 'District 8',
 'District 9',
 'District 10',
 'District 11',
 'District 12',
 'District 13',
 'District 14',
 'District 15',
 'District 16',
 'District 17',
 'District 18',
 'District 19',
 'District 20',
 'District 21',
 'District 22',
 'District 23',
 'District 24',
 'District 25',
 'District 26',
 'District 27',
 'District 28',
 'District 29',
 'District 30',
 'District 31',
 'District 32',
 'District 33',
 'District 34',
 'District 35',
 'District 36',
 'District 37',
 'District 38',
 'District 39',
 'District 40',
 'District 41',
 'District 42',
 'District 43',
 'District 44',
 'District 45',
 'District 46',
 'District 47',
 'District 48',
 'District 49',
 'District 50',
 'District 51',
 'District 52',
 'District 53',
 'District 1',
 'District 2',
 'District 3',
 'District 4',
 'District 5',
 'District 6',
 'District 7',
 'District 8',
 'District 9',
 'District 10',
 'District

In [128]:
districts = re.findall(r"District \d{1,2}.{1,300}General election candidates.+?Primary", ca, re.DOTALL)

In [129]:
len(districts)

53

In [131]:
data = pd.DataFrame()
data

""


In [183]:
def get_candidates_for_state(state):
    html = get_html_from_url("https://ballotpedia.org/United_States_House_of_Representatives_elections_in_{},_2018".format(state))
    districts = re.findall(r"District \d{1,2}.{1,300}General election candidates.+?Primary", html, re.DOTALL)
#     print(districts)
    data = pd.DataFrame()
    for district in districts:
        district_string = re.findall(r"District \d{1,2}", district)[0]
        candidate_names = re.findall(r"<a href=.+>(.+)</a>", district)
        data = data.append(pd.DataFrame({'district': [district_string]*len(candidate_names), 'name': candidate_names}), ignore_index=True)
    return data

In [185]:
get_candidates_for_state("Nevada")

,district,name
0,District 1,Dina Titus
1,District 1,Joyce Bentley
2,District 1,Daniel Garfield
3,District 1,Robert Van Strawder
4,District 1,Nevada's 1st Congressional District election (...
5,District 1,Nevada's 1st Congressional District election (...
6,District 2,Mark Amodei
7,District 2,Clint Koble
8,District 2,Nevada's 2nd Congressional District election (...
9,District 2,Nevada's 2nd Congressional District election (...


In [101]:
ex = general_candidates[0]

In [102]:
ex

'District 1</a></span></h3>\n<p><big><b>General election candidates:</b></big><br />\n<a href="/Republican_Party" title="Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Doug_LaMalfa" title="Doug LaMalfa">Doug LaMalfa</a> - Incumbent<br />\n<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Audrey_Denney" title="Audrey Denney">Audrey Denney</a>\n</p>\n<table cellspacing="0" cellpadding="0" class="multicol" style="background:transparent; width:auto;">\n<p></p>\n<tr>\n<td width="auto" align="left" valign="top">\n<p></p>\n</td>\n<td align="left" valign="top">\n<p><b><big>Primary'

In [133]:
re.findall(r"District \d{1,2}", ex)

['District 1']

In [121]:
re.findall(r"<a href.+>(.+)</a>", ex)

['Doug LaMalfa', 'Audrey Denney']

In [127]:
pd.DataFrame(['District 1']*2,['Doug LaMalfa', 'Audrey Denney'])

,0
Doug LaMalfa,District 1
Audrey Denney,District 1


In [126]:
pd.DataFrame.from_dict({'district 1': [{'name': 'Doug', 'party': 'D'}, {'name': 'Audrey', 'party': 'R'}]}, orient='index')

,0,1
district 1,"{'name': 'Doug', 'party': 'D'}","{'name': 'Audrey', 'party': 'R'}"


In [87]:
general_candidates

['District 1</a> • <a href="/California%27s_2nd_Congressional_District_election,_2018" title="California\'s 2nd Congressional District election, 2018">District 2</a> • <a href="/California%27s_3rd_Congressional_District_election,_2018" title="California\'s 3rd Congressional District election, 2018">District 3</a> • <a href="/California%27s_4th_Congressional_District_election,_2018" title="California\'s 4th Congressional District election, 2018">District 4</a> • <a href="/California%27s_5th_Congressional_District_election,_2018" title="California\'s 5th Congressional District election, 2018">District 5</a> • <a href="/California%27s_6th_Congressional_District_election,_2018" title="California\'s 6th Congressional District election, 2018">District 6</a> • <a href="/California%27s_7th_Congressional_District_election,_2018" title="California\'s 7th Congressional District election, 2018">District 7</a>• <a href="/California%27s_8th_Congressional_District_election,_2018" title="California\'s

### Make dictionary with states as keys and values are lists of dictionaries with districts as keys and lists of candidates as values